Place this code piece next to the .../etherscan/1_rest.ipynb to run

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import json

with open("../.private/keys.json") as keys_file:
    KEYS = json.load(keys_file)

APIKEY = KEYS['ETHERSCAN']
# Note: don't print the key, or if you do, delete the cell's output
# (cell outputs are saved and can be sent to Github).


In [2]:
ENDPOINT = 'https://api.etherscan.io/api'

In [3]:
# https://datagy.io/python-requests-authentication/


def process_args(params):
    queryString = "?"
    for p in params:
        queryString += p.lower() + "=" + params[p] + "&"

    queryString += "apikey=" + APIKEY
    return queryString

def etherscan(params={}):

    params['apikey'] = APIKEY
    response = requests.get(ENDPOINT,
                            headers={
                                'accept': 'application/json',
                                "User-Agent": ""
                            },
                            params=params)

    print(response)
    return response.json()

# Key takeaway

Etherscan records for 

    - all actions from an account/smart contract (as far as the action involves ETH as transaction fee), which includes
        -- time, from, to, amount, what this action is about, transaction fee, in which block it is recorded
        -- (Smart contract only) Source code
    - the current balance of a account in ETH, and the current balance for (account, token) pairs

And these are not directly available

    - the historical ETH balance of a account (only in the PRO version of the API)
    - the historical token balance of a account (only in the PRO version of the API)



And technically not feasible

    - the token transferred within an action


## Actions (=Transactions in Etherscan)



I experiment the Etherscan API with the sample account
'0x894Aa5F1E45454677A8560ddE3B45Cb5C427Ef92', or 'wombat.eth',
retrived from https://snapshot.org/#/gitcoindao.eth/proposal/0x64bc572f0b50346e8d1f300b39d02204750ec4308002794dac9247422adb27bd

Etherscan page of the address:
https://etherscan.io/address/0x894aa5f1e45454677a8560dde3b45cb5c427ef92

In [29]:
# to get the recent transactions of an address

params = {
    'module': 'account',
    'action': 'txlist',
    'address': '0x894Aa5F1E45454677A8560ddE3B45Cb5C427Ef92',
    #'toaddress': '0x894Aa5F1E45454677A8560ddE3B45Cb5C427Ef92',
    'startblock': '0',
    'endblock': '99999999'
}

# Syntax note: 'address' is mandantory and one has to input 'address' even if having 'toaddress'

res = etherscan(params)

<Response [200]>


In [30]:
df = pd.DataFrame(res['result'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   blockNumber        122 non-null    object
 1   timeStamp          122 non-null    object
 2   hash               122 non-null    object
 3   nonce              122 non-null    object
 4   blockHash          122 non-null    object
 5   transactionIndex   122 non-null    object
 6   from               122 non-null    object
 7   to                 122 non-null    object
 8   value              122 non-null    object
 9   gas                122 non-null    object
 10  gasPrice           122 non-null    object
 11  isError            122 non-null    object
 12  txreceipt_status   122 non-null    object
 13  input              122 non-null    object
 14  contractAddress    122 non-null    object
 15  cumulativeGasUsed  122 non-null    object
 16  gasUsed            122 non-null    object
 1

In [32]:
df['value'] = df['value'].astype(float)/1000000000000000000 #express in Ether
df['gasPrice'] = df['gasPrice'].astype(float)/1000000000000000000 #express in Ether
df['txnFeeETH'] = df['gasPrice'] * df['gasUsed'].astype(float)

df['timeStamp'] = df['timeStamp'].astype(int)

# reformat hash
df['from_short'] = df['from'].str[0:3] + '...' + df['from'].str[-3:]
df['to_short'] = df['to'].str[0:3] + '...' + df['to'].str[-3:]

df['time'] = df['timeStamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [33]:
vars= ['blockNumber','time','from_short','to_short','value','methodId','hash','txnFeeETH']
df[vars]

,blockNumber,time,from_short,to_short,value,methodId,hash,txnFeeETH
0,12455444,2021-05-18 01:35:07,0x9...daa,0x8...f92,0.095000,0x,0x0c31ed0cd9e81741ffeb8022a39ef45a66cde1fe52ed...,0.001260
1,12546661,2021-06-01 05:15:14,0x8...f92,0xe...564,0.010500,0xc04b8d59,0x77599fe1a04114994c4c1ead39ca786b29e0fdb9b4fb...,0.000371
2,12546714,2021-06-01 05:26:29,0x8...f92,0xe...564,0.010500,0xc04b8d59,0x9899a279a470d23e33a392b17772a25714f6d1ec346b...,0.003928
3,12591733,2021-06-08 04:26:14,0x8...f92,0x2...7f5,0.000000,0xf14fcbc8,0x819495e30f801afedfe0d26319631ef7e582c0f27480...,0.000925
4,12591750,2021-06-08 04:29:36,0x8...f92,0x2...7f5,0.004364,0xf7a16963,0x77dca0f11cec38aabf40d8e346eb420192bd581051ee...,0.004315
...,...,...,...,...,...,...,...,...
117,15454111,2022-09-01 17:15:54,0x8...f92,0xd...489,0.000000,0x15373e3d,0xfd55e3b5bee0dbba4829bdf31d8a31408af0b148007d...,0.001154
118,15517393,2022-09-11 22:42:10,0x8...f92,0xa...b48,0.000000,0x095ea7b3,0x2e63b0b106ed7958592b6d31539f54b8ca37fca66de0...,0.000471
119,15590620,2022-09-22 18:18:23,0x8...f92,0xd...eff,0.000000,0xaa77476c,0x0810b863cdfd8e73294b15b0ffded0e1c38e9d79d65f...,0.001599
120,15590625,2022-09-22 18:19:23,0x8...f92,0x6...d0f,0.000000,0x095ea7b3,0x0d08014b1e7523423acb9f30d700f58eeed5adf5a0eb...,0.000533


In [76]:
df.loc[df.hash=='0x8d583c900512dc9b2555387ca0b8cae14ac8402f1a6bd1ddec0b14c4f3da6893','to']

106    0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
Name: to, dtype: object

'Voting' is a kind of 'transaction' (or 'action'), and interestingly it seem the 'methodId' may be used to identify a 'voting' action when == '0x15373e3d', (or maybe use 'functionName')

But we have no clue on which proposal this vote is casted and for which decision (in the two examples below, we know two votes were made, but whether or not to the same proposal)

In [77]:
# https://etherscan.io/tx/0xbd23d7cc88f54fac41a56b5596b0af4a3b7693616e8c3c247b82584c4cef1ad1
df.loc[df.hash=='0xbd23d7cc88f54fac41a56b5596b0af4a3b7693616e8c3c247b82584c4cef1ad1',vars+['functionName']]

,blockNumber,time,from_short,to_short,value,methodId,hash,txnFeeETH,functionName
115,15454103,2022-09-01 17:14:45,0x8...f92,0xd...489,0.0,0x15373e3d,0xbd23d7cc88f54fac41a56b5596b0af4a3b7693616e8c...,0.001412,"castVote(uint256 proposalId, bool support)"


In [78]:
df.loc[df.hash=='0xdaf218fce0fe7e7ac36b0f77349ce0e83c5f21e009a4b39bf421b6c97e931768',vars+['functionName']]

,blockNumber,time,from_short,to_short,value,methodId,hash,txnFeeETH,functionName
116,15454108,2022-09-01 17:15:33,0x8...f92,0xd...489,0.0,0x15373e3d,0xdaf218fce0fe7e7ac36b0f77349ce0e83c5f21e009a4...,0.001358,"castVote(uint256 proposalId, bool support)"


A smart contract can be seen as both an account or a contract

example: Gitcoin: Governor Alpha

Contract address: 0xDbD27635A534A3d3169Ef0498beB56Fb9c937489

When retriving all actions related to this smart contract, the API call is identical to that for a (user) account

In [26]:
params = {
    'module': 'account',
    'action': 'txlist',
    'address': '0xDbD27635A534A3d3169Ef0498beB56Fb9c937489',
}

res = etherscan(params)

<Response [200]>


In [28]:
df = pd.DataFrame(res['result'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2444 entries, 0 to 2443
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   blockNumber        2444 non-null   object
 1   timeStamp          2444 non-null   object
 2   hash               2444 non-null   object
 3   nonce              2444 non-null   object
 4   blockHash          2444 non-null   object
 5   transactionIndex   2444 non-null   object
 6   from               2444 non-null   object
 7   to                 2444 non-null   object
 8   value              2444 non-null   object
 9   gas                2444 non-null   object
 10  gasPrice           2444 non-null   object
 11  isError            2444 non-null   object
 12  txreceipt_status   2444 non-null   object
 13  input              2444 non-null   object
 14  contractAddress    2444 non-null   object
 15  cumulativeGasUsed  2444 non-null   object
 16  gasUsed            2444 non-null   object


For smart contracts, we can extract their source codes

In [81]:
params = {
    'module': 'contract',
    'action': 'getsourcecode',
    'address': '0xDbD27635A534A3d3169Ef0498beB56Fb9c937489',
}

res = etherscan(params)

<Response [200]>


In [82]:
res

{'status': '1',
 'message': 'OK',
 'result': [{'SourceCode': '// SPDX-License-Identifier: NONE\n\npragma solidity 0.6.12;\npragma experimental ABIEncoderV2;\n\n\n\n// Part: GTCInterface\n\ninterface GTCInterface {\n    function getPriorVotes(address account, uint blockNumber) external view returns (uint96);\n}\n\n// Part: TimelockInterface\n\ninterface TimelockInterface {\n    function delay() external view returns (uint);\n    function GRACE_PERIOD() external view returns (uint);\n    function acceptAdmin() external;\n    function queuedTransactions(bytes32 hash) external view returns (bool);\n    function queueTransaction(address target, uint value, string calldata signature, bytes calldata data, uint eta) external returns (bytes32);\n    function cancelTransaction(address target, uint value, string calldata signature, bytes calldata data, uint eta) external;\n    function executeTransaction(address target, uint value, string calldata signature, bytes calldata data, uint eta) externa

## Balance

We can get the current Ether balance of an account (in wei):

(Note: I think the way of this number is calculated is = sum(TOKEN_NUMBER * TOKEN_PRICE))

In [79]:
params = {
    'module': 'account',
    'action': 'balance',
    'address': '0x894Aa5F1E45454677A8560ddE3B45Cb5C427Ef92', # address of the account
}

res = etherscan(params)

<Response [200]>


In [80]:
res

{'status': '1', 'message': 'OK', 'result': '10713051250887695955'}

We can get the current TOKEN balance of an account:

e.g. the USD coin balance of the example user 'wombat.eth'

In [88]:
params = {
    'module': 'account',
    'action': 'tokenbalance',
    'contractaddress': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', # address of smart contract of the USD coin
    'address': '0x894Aa5F1E45454677A8560ddE3B45Cb5C427Ef92', # address of the account
}

res = etherscan(params)

<Response [200]>


In [89]:
res

{'status': '1', 'message': 'OK', 'result': '7414020000'}